In [5]:
print("Okay")

Okay


In [6]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms  import CTransformers

In [3]:
#Extracting data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data ,
                     glob='*.pdf',
                    loader_cls=PyPDFLoader)
    documents =loader.load()

    return documents

In [7]:
extracted_data = load_pdf('data/')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/ramakrishnaparitala/Desktop/End-to-end-Medical-Chatbot-using-Llama2/mchatbot/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
  File "/var/folders/yv/dwndpsg104x6y4p01bk9w67r0000gn/T/ipykernel_49187/2329439660.py", line 1, in <module>
    extracted_data = load_pdf('data/')
  File "/var/folders/yv/dwndpsg104x6y4p01bk9w67r0000gn/T/ipykernel_49187/3466477432.py", line 6, in load_pdf
    documents =loader.load()
  File "/Users/ramakrishnaparitala/Desktop/End-to-end-Medical-Chatbot-using-Llama2/mchatbot/lib/python3.9/site-packages/langchain/document_loaders/directory.py", line 101, in load
    raise FileNotFoundError(f"Directory not found: '{self.path}'")
FileNotFoundError: Directory not found: 'data/'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ramakrishnaparitala/Desktop/End-to-end-Medical-Chatbot-using-Llama2/mchatbot/lib/pytho

In [7]:
# extracted_data


In [4]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [5]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

7020


In [6]:
text_chunks[0]


Document(metadata={'source': 'data/Medical_book.pdf', 'page': 1}, page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION')

In [7]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()

/var/folders/yv/dwndpsg104x6y4p01bk9w67r0000gn/T/ipykernel_18576/3136168174.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [9]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
query_result = embeddings.embed_query('Hello world')
print('Lenght ' , len(query_result))

Lenght  384


In [11]:
query_result

[-0.03447726368904114,
 0.03102319873869419,
 0.006734949070960283,
 0.0261089988052845,
 -0.03936203196644783,
 -0.16030246019363403,
 0.06692399084568024,
 -0.0064414506778120995,
 -0.047450508922338486,
 0.014758870005607605,
 0.07087535411119461,
 0.05552756413817406,
 0.01919330097734928,
 -0.02625131793320179,
 -0.010109501890838146,
 -0.026940520852804184,
 0.022307423874735832,
 -0.022226650267839432,
 -0.14969263970851898,
 -0.01749308779835701,
 0.007676230277866125,
 0.054352328181266785,
 0.003254461335018277,
 0.031725965440273285,
 -0.08462142199277878,
 -0.029405977576971054,
 0.05159567669034004,
 0.0481240376830101,
 -0.003314798232167959,
 -0.05827917903661728,
 0.04196927323937416,
 0.022210724651813507,
 0.12818880379199982,
 -0.022338908165693283,
 -0.011656265705823898,
 0.06292837113142014,
 -0.032876305282115936,
 -0.09122606366872787,
 -0.031175317242741585,
 0.052699554711580276,
 0.04703483358025551,
 -0.0842030718922615,
 -0.030056171119213104,
 -0.020744813

In [16]:
vectors = [
    {
        "id": f"chunk-{i+1}",  # Unique ID
        "values": embeddings.embed_query(chunk.page_content),  # Embedding values
        "metadata": {"content": chunk.page_content}  # Store the original text
    }
    for i, chunk in enumerate(text_chunks)
]

print(vectors[0])


{'id': 'chunk-1', 'values': [0.001746108173392713, -0.03350288048386574, -0.03290396183729172, 0.007168138865381479, -0.014603259041905403, 0.010261861607432365, -0.011515265330672264, 0.22930213809013367, -0.023232368752360344, 0.004120402503758669, -0.03656088188290596, 0.08592118322849274, 0.012972164899110794, 0.05221788212656975, -0.1023261696100235, -0.00313896918669343, -0.012686862610280514, 0.000471833860501647, -0.028485817834734917, -0.050259120762348175, 0.011550931259989738, 0.0778065025806427, 0.09282825142145157, -0.013797289691865444, -0.016935057938098907, -0.025955844670534134, -0.04956505820155144, -0.04613131657242775, 0.007290530484169722, -0.013553212396800518, 0.038439445197582245, 0.06280474364757538, 0.018353790044784546, 0.008242824114859104, 0.0017155685927718878, -0.039861809462308884, -0.011638646014034748, 0.01644628308713436, 0.025595560669898987, 0.09104609489440918, 0.029672710224986076, -0.05416032299399376, -0.045765697956085205, -0.013853823766112328

In [22]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Retrieve the Pinecone API key
api_key = os.getenv("PINECONE_API_KEY")

print(f"Pinecone API Key: {api_key}")


Pinecone API Key: 5cd2d9ee-4e85-4068-98e4-7a9b3ee4f643


In [23]:
# Now connecting with Vector DataBase : Pinecone and then storing the vectors we got above.
from pinecone import Pinecone

pc = Pinecone(api_key=api_key)
index_name = "medical-chatbot"
index = pc.Index(index_name)

# #Adding or Store the Vectors to above index
# # Upsert vectors to Pinecone
# index.upsert(vectors=vectors, namespace="ns1")

We are not send the whole Vectors at time... we will be sending in batchs so that it will be under limit

In [24]:
print(index.describe_index_stats())


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 7020}},
 'total_vector_count': 7020}


In [ ]:
from langchain.vectorstores import Pinecone
retriever = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)


# Query the index
query_vector = embeddings.embed_query("What are allergies?")
print("Query Vector:", query_vector)

response = index.query(
    vector=query_vector,
    top_k=3,
    namespace="ns1",
    include_metadata=True
)
print("Raw Pinecone Response:", response)

# Check LangChain retriever
results = retriever.similarity_search("What are allergies?")
print("LangChain Results:", results)



Query Vector: [0.025319892913103104, -0.004025820642709732, 0.029314134269952774, -0.005236376076936722, 0.06898462772369385, 0.03483610972762108, 0.0766669362783432, 0.02796238474547863, -0.05992485210299492, 0.06146517023444176, 0.04583435505628586, -0.07631244510412216, 0.018933532759547234, 0.03337517008185387, -0.0391276516020298, 0.08984296768903732, -0.03980489820241928, -0.01740577258169651, -0.0010402323678135872, -0.06232181563973427, 0.017125090584158897, 0.09288167953491211, -0.01691155880689621, -0.0016694372752681375, -0.026571134105324745, 0.06521008908748627, -0.02543237805366516, -0.0764322429895401, -0.0567757710814476, -0.040036678314208984, -0.0563729889690876, -0.02457081526517868, -0.005484601017087698, 0.03991376981139183, -0.04513121768832207, 0.0009315914940088987, -0.045972295105457306, 0.022260356694459915, 0.019632574170827866, 0.016805680468678474, -0.026513950899243355, 0.016967562958598137, -0.05027284845709801, -0.038652919232845306, -0.06781821697950363

AttributeError: 'list' object has no attribute 'replace'

[]

Now I don't have to upsert the data to Pine again after doing it once.

In [ ]:
# batch_size = 500  # Number of vectors per batch
# for i in range(0, len(vectors), batch_size):
#     batch = vectors[i:i+batch_size]
#     index.upsert(vectors=batch, namespace="ns1")
#     print(f"Upserted batch {i // batch_size + 1}")


In [19]:
import sys
batch_size = 500
# Calculate the size of a batch
batch = vectors[0:batch_size]
print(f"Batch size in bytes: {sys.getsizeof(batch)}")


Batch size in bytes: 4056


In [20]:
# Example query vector (embedding from the same model)
query_vector = embeddings.embed_query("What are allergies")

response = index.query(
    namespace="ns1",
    vector=query_vector,
    top_k=3,
    include_values=True,  # Include the embedding values if needed
    include_metadata=True  # Retrieve the stored text or metadata
)

# Print the response
print(response)



{'matches': [{'id': 'chunk-1373',
              'metadata': {'content': 'GALE ENCYCLOPEDIA OF MEDICINE 2 '
                                      '117Allergies\n'
                                      'Allergic rhinitis is commonly triggered '
                                      'by\n'
                                      'exposure to household dust, animal '
                                      'fur,or pollen. The foreign substance '
                                      'thattriggers an allergic reaction is '
                                      'calledan allergen.\n'
                                      'The presence of an allergen causes the\n'
                                      "body's lymphocytes to begin "
                                      'producingIgE antibodies. The '
                                      'lymphocytes of an allergy sufferer '
                                      'produce an unusuallylarge amount of '
                                      'IgE.\n'

In [21]:
# Access the results
for match in response["matches"]:
    text = match["metadata"]["content"]  # Retrieve the text
    score = match["score"]  # Retrieve the similarity score
    print(f"Text: {text}, Score: {score}")

Text: GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE, Score: 0.682538927
Text: allergens are the following:
• plant pollens
• animal fur and dander
• body parts from house mites (microscopic creatures
found in all houses)
• house dust• mold spores• cigarette smoke• solvents• cleaners
Common food allergens include the following:
• nuts, especially peanuts, walnuts, and brazil nuts
• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives
The following types of drugs commonly cause aller-
gic reactions:
• penicillin or 

In [34]:
prompt_template= """ 
Use the following pieces of information to answer the user's question.
If you don't know the answer , just say that you don't know , don't try to make up an answer.
Context: {context}
Question: {question}

Only return teh helpful answer below and nothing else
Helpful Answer:
"""

In [35]:
PROMPT= PromptTemplate(template= prompt_template , input_variables=['context' , 'question'])
chain_type_kwargs={'prompt': PROMPT}

In [36]:
llm = CTransformers(model ="model/llama-2-7b-chat.ggmlv3.q2_K.bin" ,
                    model_type = 'llama',
                     config= {'max_new_tokens' :512 ,
                              'temperature': 0.8} )

In [41]:
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA

# Initialize the Pinecone retriever with the embeddings
retriever = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)



In [42]:
# Create the RetrievalQA chain with the customizations
qa = RetrievalQA.from_chain_type(
    llm=llm,                      # Your LLM model (CTransformers in this case)
    retriever=retriever.as_retriever(search_kwargs={"k": 3}),  # Limit to 3 documents
    chain_type="stuff",            # Chain type, change to "stuff" or other options if necessary
    return_source_documents=True,  # Ensure source documents are returned
    chain_type_kwargs=chain_type_kwargs  # Include source document metadata
)




In [43]:
# Run the query
query = "What are allergies?"
response = qa.invoke(query)

# Print the response and the source documents
print("Response:", response['result'])  # Response answer from LLM
print("Source Documents:", response['source_documents'])  # Retrieved documents

Response: An allergy is a reaction by your immune system to a usually harmless substance called an allergen. It can cause a variety of symptoms including sneezing, itchy eyes, runny nose, stomach problems, and in severe cases, life-threatening reactions like anaphylaxiss
Allergies can occur at any age but are most common among children and young adults. Treatment consists of avoidance of the allergen and medications such as antihistamines or corticosteroids to manage symptoms.
Please let me know if you would like me to answer any other questions.
Source Documents: []


In [44]:
while True:
    user_input = input(f'Input Prompt')
    result= qa({'query': user_input})
    print("Response: " , result['result'])

/var/folders/yv/dwndpsg104x6y4p01bk9w67r0000gn/T/ipykernel_18576/3602657672.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result= qa({'query': user_input})


Response:  Acne is a common skin condition that occurs when hair follicles on the face, nose, forehead, and other areas of the body become clogged with excess oil (sebum) and dead skin cells. 
It may appear as bumps or pimples. There are different types of acne including comedones (whiteheads and blackheads), papules, pustules, nodules and in some cases, it can lead to scarring of the skin.
Acne is a chronic condition, meaning that it can persist over time unless treated. It is also important to note that acne is not caused by poor hygiene or diet, but rather by complex interactions between genetic, hormonal and environmental factors.


KeyboardInterrupt: 